In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Explorando los datos

Traemos los csv arrojados por las pruebas

In [ ]:
data_files = os.listdir('knn_results')

def load_files(filenames):
    for filename in filenames:
        yield pd.read_csv(f"knn_results/{filename}")

data = pd.concat(load_files(data_files), ignore_index=True)

data.describe()

## Agrupamiento de datos

Podemos agrupar por train size, knn y kfold para tener una idea general de como quedan los datos

In [ ]:
# podemos usar .mean() para resolver las métricas obtenidas de los distintos kfold

grouped = data.groupby(['train_size', 'knn', 'kfold']).mean()

grouped

## Análisis

¿Pero cuál es la mejor combinación?, de forma rápida podemos obtenerla así:

In [ ]:
metrics = grouped.columns.values

idx = []

for metric in metrics:
    print(f"Media más alta para {metric}")
    idx = grouped[[metric]].idxmax()[metric]
    print(grouped.loc[idx])
    print("==================")

Observemos que justamente la mejor combinación se dá en el trainset que coincide con la prueba hecha sobre los 42000 elementos del dataset con un 80% utilizados para entrenar, que son 33600 elementos. Utilizar __1-nn__ verificado utilizando cross validation __15-fold__ nos otorga la mayor media en cada métrica obtenida, en particular en el accuracy.

¿Podremos inferir alguna carácteristica sobre el sistema si fijamos el train size y ordenamos la tabla por alguna métrica? Accuracy:

In [ ]:
grouped.loc[33600].sort_values(by="accuracy", ascending=False).head(20)

Se puede observar una fuerte relación entre la cantidad de vecinos que se tiene en cuenta: mientras menos, mayor accuracy. A su vez, tiende a haber una fuerte relación con la cantidad particiones al aplicar cross validation, lo cual tiene sentido, porque a más de estas, mayor confianza hay en el sistema entrenado.

In [ ]:
def pltHeatmap(train_size, metric):
    df = grouped.loc[train_size].unstack(level=0)[metric]
    fig, ax = plt.subplots(figsize=(15,5))
    sns.heatmap(df, annot = True, ax=ax).invert_yaxis()
    plt.title(f"{metric}: train size = {train_size}")
    plt.show()

# train_sizes = pd.unique(data['train_size'])
# [ 8400 16800 25200 33600]

# metrics = grouped.columns.values
# ['accuracy' 'cohen_kappa' 'f1' 'precision' 'recall']

pltHeatmap(33600, "accuracy")

Analogamente, en este heatmap, se ve de manera muy visual la relación que se mencionaba, y si imaginamos que este es el dominio de la función para optimizar el sistema, en la region comprendida por los primeros 2x2 bloques se encuentra la mayor oportunidad para mejorar el accuracy, por lo que valdria la pena aumentar la granularidad en esa región. Notemos también que el accuracy parece ser creciente en relación a cuantas particiones se utilizan para hacer cross validation, pero decreciente en relación a la cantidad de vecinos por lo que a priori no se podría mejorar más el sistema (no se puede tener 0 vecinos) si no aumentar la confianza que tenemos en él midiendo el accuracy aumentando las particiones para validar.

In [ ]:
pltHeatmap(33600, "cohen_kappa")
pltHeatmap(33600, "f1")
pltHeatmap(33600, "precision")
pltHeatmap(33600, "recall")

Como se mencionó previamente, lo mismo sucede si miramos las demás métricas.

¿Pasará lo mismo para el accuracy con la partición más chica con la que se corrió el experimento? Train size de 8400 elementos:

In [ ]:
grouped.loc[8400].sort_values(by="accuracy", ascending=False).head(20)

Efectivamente, sucede lo mismo, pero ahora no solo notamos que la relación entre la métrica y el accuracy está dada por la cantidad de vecinos que se tome en cuenta y la cantidad de parciones para hacer cross validation, si no también por el train size, dado que al ser esta la partición más chica que disponemos con la que hicimos las pruebas, tiene también los valores más bajos para el accuracy. Veamos más graficamente dejando fija la variable que aporta a la confianza que tenemos en el sistema, la cantidad de particiones en 20:

In [ ]:
grouped = data.groupby(['kfold','knn','train_size']).mean()
df = grouped.loc[20].unstack(level=0)["accuracy"]
fig, ax = plt.subplots(figsize=(15,5))
sns.heatmap(df, annot = True, ax=ax).invert_yaxis()
plt.title(f"accuracy: kfold = 20")
plt.show()

Observemos que el accuracy disminuye no solo aumentando la cantidad de vecinos, si no también si la cantidad de elementos del train set es menor, como se había mencionado.

### ¿Por qué con 1-nn  se logran los mejores resultados? ¿Es válido para cualquier sistema?

Vimos que en todos los casos presentados del sistema, tomar 1-nn es lo mejor que podemos hacer. Hablamos de un único vecino. Respecto a la cantidad de vecinos máxima: si suponemos que las clases están balanceadas al entrenar, podemos decir entonces que tenemos aproximadamente n elementos por cada una de ellas, de esta forma, midiendo los votos de los vecinos solo por presencia (la forma más naive), bastaría con tomar 2n elementos para que la posibilidad de la mala clasificación sea más concreta ya que podria estar contando los votos de otro cluster en el espacio j-dimensional. Esto puede pasar especialmente cuando los dígitos en cuestión se parecen, como podria ser el caso del 3 y el 8, o bien el 1 y el 7. En base a esto y al sistema con el que contamos, sobre el train set de 33600 elementos que vimos previamente, tendriamos aproximadamente 3360 elementos de cada clase, con lo que utilizando 6720-nn podríamos tener un accuracy cercano a 0. (habría que probar esto).